문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv() 

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

pdf_path = '../data/콘텐츠분쟁해결_사례.pdf'

try:
    loader = PyPDFLoader(pdf_path)
    print("PDF 문서를 성공적으로 로드했습니다.")
except Exception as e:
    print(f"오류: PDF 파일을 로드할 수 없습니다. 경로를 확인하세요: {e}")
    exit()

docs = loader.load()
print(f"총 {len(docs)}페이지 로드 완료")

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF 문서를 성공적으로 로드했습니다.
총 109페이지 로드 완료


In [6]:
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

chunks = pythontext_splitter.split_documents(docs)
print(f" {len(chunks)}개 청크 생성 완료")

pythonembeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vectorstore = FAISS.from_documents(chunks, pythonembeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")
vectorstore.save_local("faiss_index_upstage")

pythonretriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

pythonllm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.2
    )

pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

prompt = PromptTemplate.from_template(pythonprompt_template)

pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,
    chain_type="stuff", 
    retriever=pythonretriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


 104개 청크 생성 완료
 FAISS 벡터스토어 생성 완료 (104개 벡터)


In [7]:

def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    else:
        return "기타"

if __name__ == "__main__":
    print("\n--- 콘텐츠 분쟁 해결 RAG 시스템 테스트 시작 ---\n")
    for i, question in enumerate(pythontest_questions, 1):
        dispute_type = classify_dispute_type(question)
        result = pythonqa_chain.invoke({"query": question})


        print(f"--- 질문 {i} ---")
        print(f"질문 내용: {result['query']}")
        print(f"분쟁 유형: {dispute_type}")
        print("\n[법률 자문 답변]")
        print(result['result'])
        print("\n[참고한 분쟁 사례]")

        for doc in result['source_documents']:
            print(f"- 페이지 {doc.metadata.get('page', 'N/A')}: \"{doc.page_content[:100]}...\"")
        print("-" * 50, "\n")


--- 콘텐츠 분쟁 해결 RAG 시스템 테스트 시작 ---

--- 질문 1 ---
질문 내용: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임

[법률 자문 답변]
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안  

#### 1. **사례 기반 분석**  
제시된 사례들을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 결과가 달라집니다:  
- **계정 소유권 확인**: 피신청인(게임사)이 계정 명의자와 실소유자를 엄격히 구분할 경우, 복구 요청이 거절될 수 있습니다(2006년 사례).  
- **오류 입증 가능성**: 시스템 오류가 객관적으로 확인될 경우 복구가 이루어졌으나(2006년 프로그램 오류 사례), 단일 사용자의 주장만으로는 인정되지 않았습니다(2009년 1,000억 게임머니 사례).  
- **게임사의 약관 및 정책**: 아이템의 소유권 귀속(게임사 또는 이용자)과 복구 절차를 명시한 약관이 결정적 역할을 합니다(2007년 해킹 아이템 회수 사례).  

---

#### 2. **법적 근거**  
- **민법 제250조(도품·유실물 특례)**:  
  - 게임 아이템이 "도품 또는 유실물"에 해당할 경우, 피해자(아이템 소유자)는 2년 이내에 반환을 청구할 수 있으나, **금전적 가치가 없는 디지털 아이템**은 적용이 제한적일 수 있습니다.  
  - 단, 게임머니 등 금전적 가치가 인정되는 경우 일부 적용 가능성 있음.  
- **전자상거래법 제17조(디지털콘텐츠 공급자의 의무)**:  
  - 시스템 오류 등 공급자 귀책 사유로 이용자에게 피해가 발생한 경우, 보상 또는 복구 의무가 발생할 수 있습니다.  
- **약관의 규제에 관한 법률 제6조(불공정 약관 무효)**:  
  - 게임사가 "모든 책임은 이용자에게 있다"는 식의 불공정 약관을 적용할 경우, 해당 조항은 무효될 수 있습